In [2]:
import glob
import pandas as pd
import numpy as np
import torch
from torchvision.transforms import ToPILImage
from IPython.display import display
import os

## Criando um DF com os file_names

In [3]:
# Pasta contendo os arquivos .npy
folder_path_1300 = 'E:/Dario_depth/depth/1300s/depth_cropped'

folder_path_1400 = 'E:/Dario_depth/depth/1400s_new/depth/val/images'

folder_path_1500 = 'E:/Dario_depth/depth/1500s_new/depth/val/images'

# Obter a lista de caminhos dos arquivos .npy usando glob
file_paths_1300 = glob.glob(folder_path_1300 + '/*.npy')

# Obter a lista de caminhos dos arquivos .npy usando glob
file_paths_1400 = glob.glob(folder_path_1400 + '/*.npy')

# Obter a lista de caminhos dos arquivos .npy usando glob
file_paths_1500 = glob.glob(folder_path_1500 + '/*.npy')

In [4]:
def get_image_ids(file_paths):

    image_ids = []


    for file_path in file_paths:
        image_id = file_path.split('\\')[-1]

        image_id = image_id.split('.')[0]

        image_ids.append(image_id)

    return image_ids


def create_df_from_image_ids(image_ids, type):

    file_names = pd.DataFrame(image_ids, columns=['image_id'])

    file_names['npy_image']    = file_names['image_id']+ '.npy'

    file_names['binary_image'] = file_names['image_id']+ '_pmask.png'

    file_names['splash_image'] = 'splash_' + file_names['image_id'] + '.png'

    file_names['type'] = type

    return file_names



def create_file_names_df(file_paths, type):

    image_ids = get_image_ids(file_paths)

    df = create_df_from_image_ids(image_ids,
                                  type = type)
    

    return df


In [5]:
file_names = create_file_names_df(file_paths_1300, 1300)

file_names = pd.concat([file_names, create_file_names_df(file_paths_1400, 1400)], ignore_index=True)

file_names = pd.concat([file_names, create_file_names_df(file_paths_1500, 1500)], ignore_index=True)

In [6]:
# write file_names
file_names.to_csv('./../utils_csv/file_names.csv', index=False)

In [5]:
file_names

,image_id,npy_image,binary_image,splash_image,type
0,calf_id_1387_20210621_743_depth_0042,calf_id_1387_20210621_743_depth_0042.npy,calf_id_1387_20210621_743_depth_0042_pmask.png,splash_calf_id_1387_20210621_743_depth_0042.png,1300
1,calf_id_1387_20210621_743_depth_0057,calf_id_1387_20210621_743_depth_0057.npy,calf_id_1387_20210621_743_depth_0057_pmask.png,splash_calf_id_1387_20210621_743_depth_0057.png,1300
2,calf_id_1387_20210621_743_depth_0059,calf_id_1387_20210621_743_depth_0059.npy,calf_id_1387_20210621_743_depth_0059_pmask.png,splash_calf_id_1387_20210621_743_depth_0059.png,1300
3,calf_id_1387_20210621_743_depth_0061,calf_id_1387_20210621_743_depth_0061.npy,calf_id_1387_20210621_743_depth_0061_pmask.png,splash_calf_id_1387_20210621_743_depth_0061.png,1300
4,calf_id_1387_20210621_743_depth_0064,calf_id_1387_20210621_743_depth_0064.npy,calf_id_1387_20210621_743_depth_0064_pmask.png,splash_calf_id_1387_20210621_743_depth_0064.png,1300
...,...,...,...,...,...
70679,calf_id_1599_20220207_408_depth_0353,calf_id_1599_20220207_408_depth_0353.npy,calf_id_1599_20220207_408_depth_0353_pmask.png,splash_calf_id_1599_20220207_408_depth_0353.png,1500
70680,calf_id_1599_20220207_408_depth_0361,calf_id_1599_20220207_408_depth_0361.npy,calf_id_1599_20220207_408_depth_0361_pmask.png,splash_calf_id_1599_20220207_408_depth_0361.png,1500
70681,calf_id_1599_20220207_408_depth_0383,calf_id_1599_20220207_408_depth_0383.npy,calf_id_1599_20220207_408_depth_0383_pmask.png,splash_calf_id_1599_20220207_408_depth_0383.png,1500
70682,calf_id_1599_20220207_408_depth_0390,calf_id_1599_20220207_408_depth_0390.npy,calf_id_1599_20220207_408_depth_0390_pmask.png,splash_calf_id_1599_20220207_408_depth_0390.png,1500


## Adicionando as pastas ao path

In [6]:
def add_prefix_based_on_type_binary(row):
    prefix = ""

    if row['type'] == 1300:
        prefix = "E:/Dario_depth/masks/1300s/binary/"
    elif row['type'] == 1400:
        prefix = "E:/Dario_depth/masks/1400s/binary/"
    elif row['type'] == 1500:
        prefix = "E:/Dario_depth/masks/1500s/binary/"

    return prefix + row['binary_image']

def add_prefix_based_on_type_npy(row):
    prefix = ""

    if row['type'] == 1300:
        prefix = 'E:/Dario_depth/depth/1300s/depth_cropped/'
    elif row['type'] == 1400:
        prefix = 'E:/Dario_depth/depth/1400s_new/depth/val/images/'
    elif row['type'] == 1500:
        prefix = 'E:/Dario_depth/depth/1500s_new/depth/val/images/'

    return prefix + row['npy_image']

In [7]:
file_names['binary_image'] = file_names.apply(add_prefix_based_on_type_binary, axis=1)
file_names['npy_image'] = file_names.apply(add_prefix_based_on_type_npy, axis=1)

## Rotacionando e salvando as binary images

A ideia é encontrar o centróide, o ponto mais longe dele e rotacionar a imagem de forma que o ponto mais longe do centróide fique na horizontal.


In [8]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist

def find_centroid(imagem_binaria):
    # Find the indices of the white pixels
    white_indices = np.where(imagem_binaria == 255)

    if len(white_indices[0]) == 0:
        return None, None  # No white pixels found

    # Calculate the centroid of the white pixels
    centroid_x = int(np.mean(white_indices[1]))
    centroid_y = int(np.mean(white_indices[0]))

    return centroid_x, centroid_y

def find_farthest_point(imagem_binaria):
    # Find the indices of the white pixels
    white_indices = np.where(imagem_binaria == 255)

    if len(white_indices[0]) == 0:
        return None  # No white pixels found

    # Calculate the centroid of the white pixels
    centroid_x = int(np.mean(white_indices[1]))
    centroid_y = int(np.mean(white_indices[0]))

    # Calculate the distances of each white pixel to the centroid
    distances_to_centroid = cdist([(centroid_x, centroid_y)], np.transpose(white_indices[::-1]))

    # Find the index of the farthest white pixel from the centroid
    farthest_index = np.argmax(distances_to_centroid)

    # Get the coordinates of the farthest point
    farthest_point = (white_indices[1][farthest_index], white_indices[0][farthest_index])

    return farthest_point


def rotate_image(imagem, angulo):
    # Obter o centro da imagem
    altura, largura = imagem.shape[:2]
    centro = (largura / 2, altura / 2)

    # Obter a matriz de rotação
    matriz_rotacao = cv2.getRotationMatrix2D(centro, angulo, 1.0)

    # Rotacionar a imagem
    imagem_rotacionada = cv2.warpAffine(imagem, matriz_rotacao, (largura, altura))

    return imagem_rotacionada



def calculate_centroid_and_farthest_point(args, save_folder):
    binary_png_path, image_id = args

    # Carregar a imagem binária
    binary_image = cv2.imread(binary_png_path, cv2.IMREAD_GRAYSCALE)

    # Encontrar o centroide
    centroid_x, centroid_y = find_centroid(binary_image)

    if centroid_x is None or centroid_y is None:
        cv2.imwrite(f'{save_folder}/rotated_{image_id}.png', binary_image)
        return None

    # Encontrar o ponto mais distante do centroide
    farthest_point = find_farthest_point(binary_image)

    if farthest_point is None:
        cv2.imwrite(f'{save_folder}/rotated_{image_id}.png', binary_image)
        return None
    
    ponto1 = (centroid_x, centroid_y)
    ponto2 = farthest_point

    # Calcular o ângulo de rotação
    angulo = np.arctan2(ponto2[1] - ponto1[1], ponto2[0] - ponto1[0]) * 180 / np.pi

    # Rotacionar a matriz resultante da multiplicação
    rotated_result = rotate_image(binary_image, angulo)

    # Salvar o resultado em um arquivo PNG
    cv2.imwrite(f'{save_folder}/rotated_{image_id}.png', rotated_result)

    return angulo

A ideia é que o código seja executado em paralelo e feito em batches de 100 imagens, para que não haja sobrecarga de memória.

In [9]:
from joblib import Parallel, delayed
from tqdm import tqdm  # Importar tqdm para a barra de progresso

# Suponha que você tenha um DataFrame chamado file_names_parte

# Preparar uma lista de tuplas com os argumentos para cada arquivo
args_list = [(row['binary_image'], row['image_id']) for _, row in file_names_parte.iterrows()]

# Definir o número de núcleos a serem usados para a paralelização
num_cores = 4  # Por exemplo, use 4 núcleos

# Definir a pasta onde os arquivos serão salvos
save_folder = 'E:/rotacionados'

batch_size = 100  # Número de arquivos a serem processados por lote

processed_count = 0  # Inicializar o contador de arquivos processados

# Criar uma lista para armazenar os resultados de ângulo de rotação e image_id
results = []

# ...

for batch_start in tqdm(range(0, len(args_list), batch_size), desc="Processing Batches", unit="batch"):
    batch_args = args_list[batch_start:batch_start + batch_size]
    
    batch_results = Parallel(n_jobs=num_cores)(
        delayed(calculate_centroid_and_farthest_point)(args, save_folder) for args in batch_args
    )
    
    # Atualizar o contador e armazenar os resultados
    processed_count += len(batch_args)
    
    # Adicionar os resultados do lote à lista de resultados
    for angulo, image_id in zip(batch_results, [args[1] for args in batch_args]):
        results.append({'image_id': image_id, 'angulo_rotacao': angulo})

# Criar um DataFrame a partir da lista de resultados
results_df = pd.DataFrame(results)

# Salvar o DataFrame em um arquivo CSV ou realizar outras operações com os resultados
results_df.to_csv('./../utils_csv/resultados_rotacao.csv', index=False)


Processed 100 files out of 70684
Processed 200 files out of 70684
Processed 300 files out of 70684
Processed 400 files out of 70684
Processed 500 files out of 70684
Processed 600 files out of 70684
Processed 700 files out of 70684
Processed 800 files out of 70684
Processed 900 files out of 70684
Processed 1000 files out of 70684
Processed 1100 files out of 70684
Processed 1200 files out of 70684
Processed 1300 files out of 70684
Processed 1400 files out of 70684
Processed 1500 files out of 70684
Processed 1600 files out of 70684
Processed 1700 files out of 70684
Processed 1800 files out of 70684
Processed 1900 files out of 70684
Processed 2000 files out of 70684
Processed 2100 files out of 70684
Processed 2200 files out of 70684
Processed 2300 files out of 70684
Processed 2400 files out of 70684
Processed 2500 files out of 70684
Processed 2600 files out of 70684
Processed 2700 files out of 70684
Processed 2800 files out of 70684
Processed 2900 files out of 70684
Processed 3000 files ou

In [10]:
results_df = pd.read_csv('./../utils_csv/resultados_rotacao.csv')

In [11]:
results_df['rotated_image'] = 'E:/rotacionados/rotated_' + results_df['image_id'] + '.png'

In [12]:
results_df

,image_id,angulo_rotacao,rotated_image
0,calf_id_1387_20210621_743_depth_0042,-13.636698,E:/rotacionados/rotated_calf_id_1387_20210621_...
1,calf_id_1387_20210621_743_depth_0057,-14.931417,E:/rotacionados/rotated_calf_id_1387_20210621_...
2,calf_id_1387_20210621_743_depth_0059,-11.768289,E:/rotacionados/rotated_calf_id_1387_20210621_...
3,calf_id_1387_20210621_743_depth_0061,-10.768145,E:/rotacionados/rotated_calf_id_1387_20210621_...
4,calf_id_1387_20210621_743_depth_0064,-17.230053,E:/rotacionados/rotated_calf_id_1387_20210621_...
...,...,...,...
70679,calf_id_1599_20220207_408_depth_0353,33.129735,E:/rotacionados/rotated_calf_id_1599_20220207_...
70680,calf_id_1599_20220207_408_depth_0361,35.013740,E:/rotacionados/rotated_calf_id_1599_20220207_...
70681,calf_id_1599_20220207_408_depth_0383,-151.986143,E:/rotacionados/rotated_calf_id_1599_20220207_...
70682,calf_id_1599_20220207_408_depth_0390,-152.034923,E:/rotacionados/rotated_calf_id_1599_20220207_...


In [13]:
def save_bounding_box(image_path, altura_bounding_box, largura_bounding_box):
    # Carregar a imagem binária
    imagem_binaria = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Encontrar o centroide da parte branca
    m = cv2.moments(imagem_binaria)

    # Se não houver pontos brancos na imagem, usar o centro como centroide
    if m["m00"] == 0:
        centroide_x = imagem_binaria.shape[1] // 2
        centroide_y = imagem_binaria.shape[0] // 2
    else:
        centroide_x = int(m["m10"] / m["m00"])
        centroide_y = int(m["m01"] / m["m00"])

    # Calcular os limites da bounding box com base no centroide
    topo = centroide_y - altura_bounding_box // 2
    base = topo + altura_bounding_box
    esquerda = centroide_x - largura_bounding_box // 2
    direita = esquerda + largura_bounding_box

    # Certificar-se de que os limites estão dentro dos limites da imagem
    topo = max(0, topo)
    base = min(imagem_binaria.shape[0], base)
    esquerda = max(0, esquerda)
    direita = min(imagem_binaria.shape[1], direita)

    # Retornar os valores de topo, base, esquerda e direita
    return topo, base, esquerda, direita


In [14]:
import cv2
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm  # Importar o tqdm para a barra de progresso

# Suponha que o DataFrame results_df tenha as colunas 'image_id' e 'binary_image'
# Defina o tamanho do batch
batch_size = 100

# Preparar uma lista de tuplas com os argumentos para cada imagem
args_list = [(row['rotated_image'], 250, 500) for _, row in results_df.iterrows()]

# Definir o número de núcleos a serem usados para a paralelização
num_cores = 4  # Por exemplo, use 4 núcleos

# Lista para armazenar os resultados
bounding_box_results = []


# Processar os batches em paralelo com barra de progresso
for batch_start in tqdm(range(0, len(args_list), batch_size), total=len(args_list)//batch_size):
    batch_args = args_list[batch_start:batch_start + batch_size]
    
    batch_results = Parallel(n_jobs=num_cores)(
        delayed(save_bounding_box)(*args) for args in batch_args
    )
    
    for result, row in zip(batch_results, results_df.iloc[batch_start:batch_start + batch_size].iterrows()):
        image_id = row[1]['image_id']
        topo, base, esquerda, direita = result
        
        bounding_box_results.append({
            'image_id': image_id,
            'topo': topo,
            'base': base,
            'esquerda': esquerda,
            'direita': direita
        })

# Criar um DataFrame a partir dos resultados da bounding box
bounding_box_df = pd.DataFrame(bounding_box_results)

# Salvar o DataFrame em um arquivo CSV ou realizar outras operações com os resultados
bounding_box_df.to_csv('./../utils_csv/resultados_bounding_box.csv', index=False)

707it [03:01,  3.90it/s]                         


In [15]:
bounding_box_df

,image_id,topo,base,esquerda,direita
0,calf_id_1387_20210621_743_depth_0042,107,357,234,734
1,calf_id_1387_20210621_743_depth_0057,115,365,216,716
2,calf_id_1387_20210621_743_depth_0059,112,362,217,717
3,calf_id_1387_20210621_743_depth_0061,110,360,220,720
4,calf_id_1387_20210621_743_depth_0064,120,370,221,721
...,...,...,...,...,...
70679,calf_id_1599_20220207_408_depth_0353,122,372,292,792
70680,calf_id_1599_20220207_408_depth_0361,109,359,285,785
70681,calf_id_1599_20220207_408_depth_0383,125,375,51,551
70682,calf_id_1599_20220207_408_depth_0390,117,367,55,555


In [16]:
# Realizar o merge entre bounding_box_df e results_df com base na coluna 'image_id'
merged_df = results_df.merge(bounding_box_df, on='image_id', how='inner')

# Salvar o DataFrame resultante em um arquivo CSV
merged_df.to_csv('./../utils_csv/merged_results.csv', index=False)